# Document Classification with Naive Bayes - Lab

## Introduction

In this lesson, you'll practice implementing the Naive Bayes algorithm on your own.

## Objectives

In this lab you will:  

* Implement document classification using Naive Bayes

## Import the dataset

To start, import the dataset stored in the text file `'SMSSpamCollection'`.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Your code here
df = pd.read_csv("SMSSpamCollection", sep="\t", names=['label', 'text'])
df.head(20)

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


## Account for class imbalance

To help your algorithm perform more accurately, subset the dataset so that the two classes are of equal size. To do this, keep all of the instances of the minority class (spam) and subset examples of the majority class (ham) to an equal number of examples.

In [3]:
# Your code here
spam_set = df[df["label"]=="spam"]
ham_set = df[df["label"]=="ham"].sample(len(spam_set))

df2 = pd.concat([spam_set, ham_set])
df2["label"].value_counts()

spam    747
ham     747
Name: label, dtype: int64

In [4]:
p_classes = dict(df2["label"].value_counts(normalize=True))

## Train-test split

Now implement a train-test split on the dataset: 

In [5]:
# Your code here
from sklearn.model_selection import train_test_split
X = df2["text"]
y = df2["label"]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=17)
train_df = pd.concat([X_train, y_train], 1)
test_df = pd.concat([X_test, y_test], 1)

## Create the word frequency dictionary for each class

Create a word frequency dictionary for each class: 

In [6]:
# Your code here
class_word_freq = {} 
classes = train_df["label"].unique()
for class_ in classes:
    temp_df = train_df[train_df["label"] == class_]
    bag = {}
    for row in temp_df.index:
        doc = temp_df["text"][row]
        for word in doc.split():
            bag[word] = bag.get(word, 0) + 1
    class_word_freq[class_] = bag

## Count the total corpus words
Calculate V, the total number of words in the corpus: 

In [7]:
# Your code here
vocabulary = set()
for text in train_df["text"]:
    for word in text.split():
        vocabulary.add(word)
V = len(vocabulary)
V

6098

## Create a bag of words function

Before implementing the entire Naive Bayes algorithm, create a helper function `bag_it()` to create a bag of words representation from a document's text.

In [8]:
# Your code here
def bag_it(text):
    bag = {}
    for word in text.split():
        bag[word] = bag.get(word, 0) + 1
    return bag

## Implementing Naive Bayes

Now, implement a master function to build a naive Bayes classifier. Be sure to use the logarithmic probabilities to avoid underflow.

In [9]:
# Your code here
def classify_doc(doc, class_word_freq, p_classes, V, return_posteriors=False):
    bag = bag_it(doc)
    classes = []
    posteriors = []
    for class_ in class_word_freq.keys():
        p = np.log(p_classes[class_])
        for word in bag.keys():
            numerator = bag[word] + 1
            denominator = class_word_freq[class_].get(word, 0) + V
            p += np.log(numerator/denominator)
        classes.append(class_)
        posteriors.append(p)
    if return_posteriors:
        print(posteriors)
    return classes[np.argmax(posteriors)]

## Test your classifier

Finally, test your classifier and measure its accuracy. Don't be perturbed if your results are sub-par; industry use cases would require substantial additional preprocessing before implementing the algorithm in practice.

In [10]:
# Your code here
y_hat_train = X_train.map(lambda x: classify_doc(x, class_word_freq, p_classes, V))
residuals = y_train == y_hat_train
residuals.value_counts(normalize=True)

False    0.760714
True     0.239286
dtype: float64

## Level up (Optional)

Rework your code into an appropriate class structure so that you could easily implement the algorithm on any given dataset.

## Summary

Well done! In this lab, you practiced implementing Naive Bayes for document classification!